In [1]:
import geopandas as gpd
from geopandas import GeoDataFrame

import pandas as pd
from shapely.geometry import Point,LineString
import networkx as nx
from shapely.ops import split, snap
from shapely.geometry import Point, LineString
import numpy as np
from multiprocessing import Pool
from tqdm import tqdm


In [ ]:
# Read the road dataset
roads = gpd.read_file(r"C:\Users\bsf31\Documents\post-meds\data\signal\tl_2023_06083_roads_sbco.gpkg")
# Dissolve the road lines to merge connected segments
dissolved_roads = roads.dissolve(by='LINEARID') 


# Intersections & Deadends

In [ ]:
# Extract the start and end points of each line
start_points = dissolved_roads.geometry.apply(lambda x: Point(x.coords[0]))
end_points = dissolved_roads.geometry.apply(lambda x: Point(x.coords[-1]))
# Concatenate the start and end points into a single GeoSeries using pd.concat()
all_points = gpd.GeoSeries(pd.concat([start_points, end_points]), crs=roads.crs)
# Count the occurrence of each point
point_counts = all_points.value_counts()


# Filter the points that appear only once (intersections & deadends)
intersections_deadends = point_counts[point_counts == 1]
# Create a dictionary with the dead end points and their counts
intersections_deadends_data = {'geometry': intersections_deadends.index, 'count': intersections_deadends.values}
 #Create a GeoDataFrame with the dead end points
intersections_deadends_data_points = gpd.GeoDataFrame(intersections_deadends_data, crs=roads.crs)
# Save the dead end points to a new shapefile
intersections_deadends_data_points.to_file(r"C:\Users\bsf31\Documents\post-meds\data\signal\sbco_intersection_deadends.gpkg", driver='GPKG')


# Deadends
## converts a dataset of roads into a graph representation and identifies dead-end points within that network.

* Graph Initialization: The process begins with the initialization of a graph. In this context, a graph is a mathematical structure used to model pairwise relations between objects. Here, roads and intersections are modeled as edges and nodes, respectively.
* Geometry Simplification and Conversion: Each road segment (line geometry) from the dataset is simplified to remove minor irregularities and potential complexities. This simplification helps in avoiding inaccuracies when converting these geometries into a graph format. The simplified line geometries are then converted into a series of points (nodes in the graph), and consecutive points are connected by edges. This effectively builds the network where paths (edges) between points (nodes) represent segments of the road.
* Identification of Dead Ends: Once the graph is built, the code identifies dead ends in the network. A dead end is recognized by having a node with only one connected edge (degree of 1). This means there is no alternate path from that node, signifying the end of a road without any junctions.

In [ ]:
roads_dissolved = roads.dissolve(by='LINEARID') 
# Initialize a new graph. This will be used to store the road network where nodes represent junctions or endpoints of roads, and edges represent the roads themselves.
G = nx.Graph()

# Iterate over each row in the dissolved roads GeoDataFrame. Each row corresponds to a road segment.
for index, row in roads_dissolved.iterrows():
    # Simplify the geometry of the road segment to remove minor variations and potential self-intersections. This is important for accurately converting the geometries into a graph structure without unnecessary complexity.
    line = row.geometry.simplify(tolerance=0.01)
    
    # Check if the simplified geometry is a LineString (a simple line shape). We only want to process line shapes as these represent the roads.
    if isinstance(line, LineString):
        # Convert the line's coordinates into a list of tuples. Each tuple represents a point (node) on the line.
        points = list(map(tuple, line.coords))
        
        # Add these points as a path in the graph. This connects each consecutive pair of points with an edge, effectively building the road network in graph form.
        nx.add_path(G, points)

# Identify all nodes in the graph that have a degree of 1. A node with a degree of 1 only has one connection (edge) to another node, which means it is an endpoint without any further continuation - a dead end.
dead_ends = [node for node, degree in G.degree() if degree == 1]

# Convert the list of dead end nodes (points) back into a GeoDataFrame for further spatial analysis. This allows us to use geographic functions on the points.
dead_ends_gdf = gpd.GeoDataFrame(geometry=gpd.points_from_xy([p[0] for p in dead_ends], [p[1] for p in dead_ends]), crs=roads.crs)


In [ ]:
# Save the dead end points to a new shapefile
dead_ends_gdf.to_file(r"C:\Users\bsf31\Documents\post-meds\data\signal\sbco_deadends.gpkg", driver='GPKG')

# Intersections

In [ ]:
# Perform a spatial difference
 #This operation returns the geometric difference of points in intersections_deadends that are not in dead_end_points.
true_intersections = gpd.overlay(intersections_deadends_data_points, dead_ends_gdf, how='difference')

# Now true_intersections contains only the intersection points that are not dead ends.
# Optionally, save to file
true_intersections.to_file(r"C:\Users\bsf31\Documents\post-meds\data\signal\sbco_intersections.gpkg", driver='GPKG')

# QGIS
* Using roads layer use ***Split with line*** using roads twice
* using  '***Extract specific vertices***' with 0, -1 as input on 'Split' layer, produces 'Vertices'
* Take difference between 'Vertices' and 'sbco_deadends' produced from this notebook, produces 'Difference'
* Using 'Difference' again take difference using 'true_intersection' produced from this notebook, this produces another 'Differece' layer which is the remaining intersections not captured in this code.
* edit attribute table with field calculator, existing column 'fid', @row_number
Result is dependent on quality of original road layer. REQUIRES manual inspection, especially in long isolated roads and in urban areas with many intersections. In long roads a random vertix can be sometimes found that ties one road together. In urban areas with high number of intersections the many vertices are harder to capture. 

If 100% coverage of intersections is desired, — this analysis doesn't absolutely require it as the roads that are missing intersections reach an intersection or deadend — then the reccomendation is that after manual inspection and removal of unwanted points, merge the three layers of sbco_deadends, sbco_intersections, and the final difference layer. In qgis use the extract all vertices tool and take the difference with the merged layer. This will produce an extreme amount of points, create a blank shapefile, cut and paste the intersection points desired manually.

Using simplify on roads  may help reduce when extracting all vertices.

In [ ]:
missing_vertices = gpd.read_file(r"C:\Users\bsf31\Documents\post-meds\data\signal\sbco_missing_vertices.gpkg")

In [ ]:
# Check for NaN coordinates in each geometry
valid_points = missing_vertices[~missing_vertices.geometry.apply(lambda geom: np.isnan(geom.x) or np.isnan(geom.y))]


In [ ]:
valid_points['type'] = 'intersection'

In [ ]:
valid_points

In [ ]:
valid_points = valid_points[['type', 'geometry']]

In [ ]:
valid_points

In [ ]:
valid_points.to_file(r"C:\Users\bsf31\Documents\post-meds\data\signal\valid_vertices.gpkg", driver='GPKG')

In [ ]:

filepath = r"C:\Users\bsf31\Documents\post-meds\data\signal\sbco_deadends.gpkg"
sbco_deadends = gpd.read_file(filepath)


filepath = r"C:\Users\bsf31\Documents\post-meds\data\signal\sbco_intersections.gpkg"
sbco_intersections = gpd.read_file(filepath)


In [ ]:
sbco_deadends['type'] = 'deadend'
sbco_deadends = sbco_deadends[['type', 'geometry']]
sbco_deadends

In [ ]:
sbco_intersections['type'] = 'intersection'
sbco_intersections = sbco_intersections[['type', 'geometry']]
sbco_intersections

In [ ]:
deadends_intersections = gpd.GeoDataFrame(pd.concat([sbco_deadends, sbco_intersections, valid_points], ignore_index=True))


In [ ]:
deadends_intersections.to_file(r"C:\Users\bsf31\Documents\post-meds\data\signal\clean_deadends_intersections.gpkg", driver='GPKG')

In [ ]:
rwmp = gpd.read_file(r"C:\Users\bsf31\Documents\post-meds\data\signal\RWMP_AREA_2229.gpkg")

In [ ]:
rwmp_deadends_intersections = gpd.clip(deadends_intersections, rwmp)

In [ ]:
rwmp_deadends_intersections.to_file(r"C:\Users\bsf31\Documents\post-meds\data\signal\rwmp_deadends_intersections.gpkg", driver='GPKG')

# Single Egress Points

In [ ]:
rmwp_road = gpd.read_file(r"C:\Users\bsf31\Documents\post-meds\data\signal\rwmp_road_split.gpkg")


In [ ]:
roads = gpd.read_file(r"C:\Users\bsf31\Documents\post-meds\data\signal\tl_2023_06083_roads_sbco_split.gpkg")


In [ ]:
# Filter points to get intersections and deadends
intersections = deadends_intersections[deadends_intersections['type'] == 'intersection']
deadends = deadends_intersections[deadends_intersections['type'] == 'deadend']





In [ ]:
# Initialize list to hold single egress roads
single_egress_roads = []

# Loop through each road in the dataset
for index, road in roads.iterrows():
    # Check if the road intersects with dead end points
    intersects_deadends = deadends.intersects(road.geometry)
    if sum(intersects_deadends) == 1:  # Only one dead end
        single_egress_roads.append(road)

# Convert list to GeoDataFrame
single_egress_roads = gpd.GeoDataFrame(single_egress_roads, columns=roads.columns, crs=roads.crs)



In [ ]:
single_egress_roads

In [ ]:
single_egress_roads.to_file(r"C:\Users\bsf31\Documents\post-meds\data\signal\single_egress_roads_sbco.gpkg", driver='GPKG')

In [ ]:
#  Identify intersection points for single egress roads
def find_intersections(egress_roads, all_intersections):
    # Properly initialize an empty GeoDataFrame with a geometry column
    egress_points = gpd.GeoDataFrame(columns=['geometry'], crs=egress_roads.crs)
    
    for road in egress_roads.geometry:
        # Buffer the road slightly to ensure all potential intersections are captured
        road_buffer = road.buffer(5)  # Adjust the buffer size as needed based on your spatial unit
        # Find intersections that are within the road buffer
        intersecting_points = all_intersections[all_intersections.intersects(road_buffer)]
        # Concatenate current egress_points with new intersecting_points
        egress_points = pd.concat([egress_points, intersecting_points], ignore_index=True)

    # Ensure the result is a GeoDataFrame with the correct CRS
    egress_points = gpd.GeoDataFrame(egress_points, geometry='geometry', crs=egress_roads.crs)
    
    return egress_points.drop_duplicates(subset='geometry')



In [ ]:
egress_points = find_intersections(single_egress_roads, intersections)

In [ ]:
egress_points = egress_points[['type', 'geometry']]
egress_points

In [ ]:
egress_points.to_file(r"C:\Users\bsf31\Documents\post-meds\data\signal\egress_points_sbco.gpkg", driver='GPKG')

In [ ]:
# Convert intersection points to a MultiPoint object (necessary for the split operation)
intersection_points = intersections.geometry.unary_union


In [ ]:
def split_roads(road, points):
    if points.intersects(road.geometry):
        return split(road.geometry, points)
    else:
        return [road.geometry]

# Apply the split function to each road segment
split_roads_layer = rmwp_road.copy()
split_roads_layer['geometry'] = split_roads_layer.apply(lambda x: split_roads(x, intersection_points), axis=1)


In [ ]:
# Since split can create multiple geometries, you need to 'explode' them into separate rows
split_roads_layer = split_roads_layer.explode().reset_index(drop=True)


In [ ]:
split_roads_layer = gpd.GeoDataFrame(split_roads_layer, crs=rmwp_road.crs)


In [ ]:
split_roads_layer.to_file(r"C:\Users\bsf31\Documents\post-meds\data\signal\split_roads_layer.gpkg", driver='GPKG')

In [ ]:
# # Identify nodes that, when removed, increase the number of connected components - these nodes represent single egress points.
# cut_points = list(nx.articulation_points(G))

# # Convert these nodes back to a GeoDataFrame for spatial analysis
# cut_points_gdf = gpd.GeoDataFrame(geometry=gpd.points_from_xy([p[0] for p in cut_points], [p[1] for p in cut_points]))

In [ ]:
# # Assuming roads_dissolved is your GeoDataFrame of roads
# G = nx.Graph()

# # Convert line geometries to graph, focusing only on true endpoints
# for line in roads_dissolved.geometry:
#     if isinstance(line, LineString):
#         # Simplify the line to minimize unnecessary intermediate points
#         simplified_line = line.simplify(tolerance=0.01)
#         endpoints = [simplified_line.coords[0], simplified_line.coords[-1]]
#         nx.add_path(G, endpoints)

# # Use DBSCAN or similar to cluster endpoints to mitigate multiple close points issue
# coords = [point for point, degree in G.nodes(data=True) if G.degree(point) == 1]
# db = DBSCAN(eps=10, min_samples=1).fit(coords)  # Adjust eps based on your spatial resolution needs
# clusters = db.labels_

# # Convert clustered endpoints back to a GeoDataFrame
# clustered_points = [MultiPoint([coords[i] for i in range(len(coords)) if clusters[i] == k]).centroid for k in set(clusters)]
# dead_ends_gdf = gpd.GeoDataFrame(geometry=gpd.points_from_xy([p.x for p in clustered_points], [p.y for p in clustered_points]))


In [ ]:
# Identifying articulation points and analyzing their impact
articulation_points = list(nx.articulation_points(G))
impact_dict = {}

# Analyzing the size of components when an articulation point is removed
for point in articulation_points:
    G_temp = G.copy()
    G_temp.remove_node(point)
    components = list(nx.connected_components(G_temp))
    # Find the largest component size when the point is removed
    largest_component_size = max([len(comp) for comp in components])
    total_size = len(G_temp.nodes())
    isolated_size = total_size - largest_component_size
    # Store the impact information
    impact_dict[point] = isolated_size

# Filtering to find significant single egress points
significant_points = {k: v for k, v in impact_dict.items() if v > 50}  # adjust 50 to your specific threshold

# Convert significant articulation points back to GeoDataFrame
significant_points_gdf = gpd.GeoDataFrame(
    geometry=gpd.points_from_xy([p[0] for p in significant_points.keys()], [p[1] for p in significant_points.keys()]),
    crs=roads.crs
)


In [ ]:
# Save the dead end points to a new shapefile
significant_points_gdf.to_file(r"C:\Users\bsf31\Documents\post-meds\data\signal\sbco_single_egress_points2.gpkg", driver='GPKG')

In [ ]:
# # Extract the start and end points of each line
# start_points = dissolved_roads.geometry.apply(lambda x: Point(x.coords[0]))
# end_points = dissolved_roads.geometry.apply(lambda x: Point(x.coords[-1]))
# # Concatenate the start and end points into a single GeoSeries using pd.concat()
# all_points = gpd.GeoSeries(pd.concat([start_points, end_points]), crs=roads.crs)
# # Count the occurrence of each point
# point_counts = all_points.value_counts()


# # Filter the points that appear only once (dead ends)
# dead_ends = point_counts[point_counts == 1]
# # Create a dictionary with the dead end points and their counts
# dead_end_data = {'geometry': dead_ends.index, 'count': dead_ends.values}
#  #Create a GeoDataFrame with the dead end points
# dead_end_points = gpd.GeoDataFrame(dead_end_data, crs=roads.crs)
# # Save the dead end points to a new shapefile
# dead_end_points.to_file(r"C:\Users\bsf31\Documents\post-meds\data\signal\sbco_deadends.gpkg", driver='GPKG')
# #2

# # Create a spatial index for the road lines
# spatial_index = dissolved_roads.sindex

# # Function to check if a point is a dead end
# def is_dead_end(point):
#     # Create a small buffer around the point
#     buffer_dist = 1e-5  # Adjust the buffer distance as needed
#     buffer = point.buffer(buffer_dist)
    
#     # Find the indices of the lines that intersect the buffer
#     possible_matches_index = list(spatial_index.intersection(buffer.bounds))
#     possible_matches = dissolved_roads.iloc[possible_matches_index]
    
#     # Check if the point touches exactly one line geometry
#     touching_lines = [line for line in possible_matches.geometry if point.touches(line)]
    
#     return len(touching_lines) == 1
# # Function to check if a point is a dead end
# def is_dead_end(point):
#     # Create a small buffer around the point
#     buffer_dist = 1e-8  # Adjust the buffer distance as needed
#     buffer = point.buffer(buffer_dist)
    
#     # Find the indices of the lines that intersect the buffer
#     possible_matches_index = list(spatial_index.intersection(buffer.bounds))
#     possible_matches = dissolved_roads.iloc[possible_matches_index]
    
#     # Check if the point touches any line geometries
#     touches_line = any(point.touches(line) for line in possible_matches.geometry)
    
#     return not touches_line
 

# # Identify the dead end points
# dead_ends2 = all_points[all_points.apply(is_dead_end)]
# # Create a GeoDataFrame with the dead end points
# dead_end_points2 = gpd.GeoDataFrame(geometry=dead_ends2, crs=roads.crs)


# # Read the road dataset
# roads = gpd.read_file(r"C:\Users\bsf31\Documents\post-meds\data\signal\tl_2023_06083_roads_sbco.gpkg")
# # Dissolve the road lines to merge connected segments
# dissolved_roads = roads.dissolve(by='LINEARID') 
# # Reset the index of dissolved_roads
# dissolved_roads = dissolved_roads.reset_index()
# # Extract the start and end points of each line
# start_points = dissolved_roads.geometry.apply(lambda x: Point(x.coords[0]))
# end_points = dissolved_roads.geometry.apply(lambda x: Point(x.coords[-1]))
# # Concatenate the start and end points into a single GeoSeries
# all_points = pd.concat([start_points, end_points])

# # Create a DataFrame where LINEARID is repeated for each start and end point
# repeated_linearid = dissolved_roads.loc[dissolved_roads.index.repeat(2), 'LINEARID'].reset_index(drop=True)
# # Create a GeoDataFrame with all points and their corresponding LINEARID
# all_points_gdf = gpd.GeoDataFrame({'geometry': all_points, 'LINEARID': repeated_linearid})

# # Count the number of occurrences of each LINEARID for each point
# linearid_counts = all_points_gdf.groupby(all_points_gdf.geometry)['LINEARID'].count()

# # Identify the dead end points (points with only one occurrence of LINEARID)
# dead_ends = linearid_counts[linearid_counts == 1].index

# # Create a GeoDataFrame with the dead end points
# dead_end_points = gpd.GeoDataFrame(geometry=dead_ends, crs=roads.crs)

# # Save the dead end points to a new shapefile
# dead_end_points.to_file(r"C:\Users\bsf31\Documents\post-meds\data\signal\sbco_deadends.gpkg", driver='GPKG')
# # Load the road data
# roads = gpd.read_file(r"C:\Users\bsf31\Documents\post-meds\data\signal\tl_2023_06083_roads_sbco.gpkg")



# Road  Order System

In [2]:
# Read the layers as GeoDataFrames
road_order_data = r"C:\Users\bsf31\Documents\post-meds\data\signal\road_order\road_order.gpkg"

In [ ]:
clean = gpd.read_file(r"C:\Users\bsf31\Documents\post-meds\data\signal\road_order\base\clean_deadends_intersections.gpkg")

In [3]:
dead_ends =  gpd.read_file(road_order_data, layer = 'sbco_deadends')
egress_points = gpd.read_file(road_order_data, layer='egress_points_sbco')


In [ ]:
remove_dead = clean.overlay(dead_ends, how = 'symmetric_difference')
remove_egress = remove_dead.overlay(egress_points, how = 'symmetric_difference')

In [ ]:
remove_egress.rename(columns = {'egress_order_1': 'egress_order'}, inplace = True)
remove_egress.drop(columns = ['egress_order_2', 'type_2'], inplace = True)


In [ ]:
remove_egress.to_file(road_order_data,layer = 'intersections_wo_egresspoints', driver = 'GPKG')

In [4]:


intersections = gpd.read_file(road_order_data, layer= 'intersections_wo_egresspoints')
roads = gpd.read_file(road_order_data, layer = 'all_wo_single_egress_roads')
single_egress_roads = gpd.read_file(road_order_data, layer = 'single_egress_roads_sbco')


In [ ]:
dead_ends

In [ ]:
egress_points

In [ ]:
intersections

In [ ]:
roads

In [ ]:
single_egress_roads

In [ ]:


def assign_egress_order(intersection_layer, road_layer, egress_point_layer, single_egress_road_layer):

    # Create a dictionary to store the egress_order values for each feature ID
    egress_order_dict = {}

    # Create a NetworkX graph from the road layer
    G = nx.Graph()
    for _, road in roads.iterrows():
        if road.geometry.type == 'MultiLineString':
            for line in road.geometry.geoms:
                start_node = line.coords[0]
                end_node = line.coords[-1]
                G.add_edge(start_node, end_node, geometry=line, fid=tuple(road.index))
        else:
            start_node = road.geometry.coords[0]
            end_node = road.geometry.coords[-1]
            G.add_edge(start_node, end_node, geometry=road.geometry, fid=tuple(road.index))

    # Assign egress_order values to intersection layer
    for _, intersection in intersections.iterrows():
        intersection_id = tuple(intersection.index)
        intersection_geom = intersection.geometry

        # Find the connected road segments
        connected_roads = []
        for _, road in roads.iterrows():
            if road.geometry.intersects(intersection_geom):
                connected_roads.append(road)

        # Find the highest egress_order value among the connected roads
        max_egress_order = 0
        for road in connected_roads:
            road_id = tuple(road.index)
            if road_id in egress_order_dict:
                max_egress_order = max(max_egress_order, egress_order_dict[road_id])

        # Assign the highest egress_order value to the intersection
        egress_order_dict[intersection_id] = max_egress_order

    # Assign egress_order values to road layer
    for _, road in roads.iterrows():
        road_id = tuple(road.index)
        road_geom = road.geometry

        # Check if the road is connected to an egress point
        for _, egress_point in egress_points.iterrows():
            if road_geom.intersects(egress_point.geometry):
                egress_order_dict[road_id] = 3
                break

        # Check if the road is a single egress road
        for _, single_egress_road in single_egress_roads.iterrows():
            if road_geom.equals(single_egress_road.geometry):
                egress_order_dict[road_id] = 2
                break

        # Check if the road is connected to an intersection
        if road_id not in egress_order_dict:
            for _, intersection in intersections.iterrows():
                if road_geom.intersects(intersection.geometry):
                    intersection_id = tuple(intersection.index)
                    if intersection_id in egress_order_dict:
                        egress_order_dict[road_id] = egress_order_dict[intersection_id]
                        break

    # Update the egress_order values in the intersection layer
    intersections['egress_order'] = intersections.index.map(lambda x: egress_order_dict.get(tuple(x), None))

    # Update the egress_order values in the road layer
    roads['egress_order'] = roads.index.map(lambda x: egress_order_dict.get(tuple(x), None))

    return intersections, roads





In [ ]:
# Call the function to assign egress_order values
updated_intersections, updated_roads = assign_egress_order(intersections, roads, egress_points, single_egress_roads)



In [ ]:
# Save the updated layers to new shapefiles
updated_intersections.to_file('path/to/updated_intersection_layer.shp')
updated_roads.to_file('path/to/updated_road_layer.shp')

In [ ]:
def create_graph(road):
    road_data = road[1]  # Access the row data
    if road_data.geometry.type == 'MultiLineString':
        lines = [(line.coords[0], line.coords[-1], {'geometry': line, 'fid': road_data.name}) for line in road_data.geometry.geoms]
    else:
        lines = [(road_data.geometry.coords[0], road_data.geometry.coords[-1], {'geometry': road_data.geometry, 'fid': road_data.name})]
    return lines

def assign_egress_order(intersections, roads, egress_points, single_egress_roads):
    # Create a dictionary to store the egress_order values for each feature ID
    egress_order_dict = {}

    # Create a NetworkX graph from the road layer
    with Pool() as pool:
        edges = pool.map(create_graph, roads.iterrows())
        G = nx.Graph()
        G.add_edges_from([edge for sublist in edges for edge in sublist])

    # Assign egress_order values to intersection layer
    def assign_intersection_order(intersection):
        intersection_id = intersection.name
        intersection_geom = intersection.geometry

        # Find the connected road segments
        connected_roads = roads[roads.geometry.intersects(intersection_geom)]

        # Find the highest egress_order value among the connected roads
        max_egress_order = connected_roads.apply(lambda x: egress_order_dict.get(x.name, 0)).max()

        # Assign the highest egress_order value to the intersection
        egress_order_dict[intersection_id] = max_egress_order

    intersections.apply(assign_intersection_order, axis=1)

    # Assign egress_order values to road layer
    def assign_road_order(road):
        road_id = road.name
        road_geom = road.geometry

        # Check if the road is connected to an egress point
        if road_geom.intersects(egress_points.unary_union):
            egress_order_dict[road_id] = 3
            return

        # Check if the road is a single egress road
        if road_geom.equals(single_egress_roads.unary_union):
            egress_order_dict[road_id] = 2
            return

        # Check if the road is connected to an intersection
        if road_id not in egress_order_dict:
            connected_intersections = intersections[intersections.geometry.intersects(road_geom)]
            if not connected_intersections.empty:
                intersection_id = connected_intersections.iloc[0].name
                if intersection_id in egress_order_dict:
                    egress_order_dict[road_id] = egress_order_dict[intersection_id]

    roads.apply(assign_road_order, axis=1)

    # Update the egress_order values in the intersection layer
    intersections['egress_order'] = intersections.index.map(egress_order_dict)

    # Update the egress_order values in the road layer
    roads['egress_order'] = roads.index.map(egress_order_dict)

    return intersections, roads


In [ ]:
updated_intersections, updated_roads = assign_egress_order(intersections, roads, egress_points, single_egress_roads)
